In [1]:
import gzip
import json
import pandas as pd

In [3]:
# Read dataset0.json.gz, dataset1.json.gz, dataset2.json.gz ==============================================================
objs=[]
with gzip.open(r"/Users/tangyating/desktop/Group-Project/dataset0.json.gz", 'rt') as f:
    for line in f:
        objs.append(json.loads(line))

# Read data.info table ====================================================================================================
data = pd.read_csv(r"/Users/tangyating/desktop/Group-Project/data.info")
data = pd.DataFrame(data)


rows = []
for i in range(len(objs)):
    for key1, inner_dict in objs[i].items():
        for key2, inner_list in inner_dict.items():
            for key3, sublist in inner_list.items():
                for subsublist in sublist:
                    rows.append([key1, key2, key3] + subsublist)


df = pd.DataFrame(rows, columns=["transcript_id", "transcript_position", "combined nucleotides", "dwelling_time1", "sd1", "mean1", "dwelling_time2", "sd2", "mean2", "dwelling_time3", "sd3", "mean3"])

df['transcript_position'] = df['transcript_position'].astype(int)
data['transcript_position'] = data['transcript_position'].astype(int)

df = pd.merge(df, data, on=['transcript_id', 'transcript_position'], how='inner')

#df.to_csv('df.csv')


In [4]:
df

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,gene_id,label
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11027101,ENST00000641834,1693,TTGACAT,0.00418,7.49,108.0,0.00564,10.20,116.0,0.01000,2.01,76.4,ENSG00000167747,0
11027102,ENST00000641834,1693,TTGACAT,0.00664,1.91,109.0,0.00598,12.30,110.0,0.01760,2.61,74.6,ENSG00000167747,0
11027103,ENST00000641834,1693,TTGACAT,0.00721,4.58,105.0,0.00398,6.58,113.0,0.00316,2.28,85.3,ENSG00000167747,0
11027104,ENST00000641834,1693,TTGACAT,0.00266,2.33,109.0,0.00913,10.40,108.0,0.00664,4.44,76.8,ENSG00000167747,0


In [2]:
df = pd.read_csv('/Users/tangyating/downloads/df.csv')

agg_functions = {
    'gene_id': 'first',
    'combined nucleotides': 'first',
    'dwelling_time1': 'mean',
    'sd1': 'mean',
    'mean1': 'mean',
    'dwelling_time2': 'mean',
    'sd2': 'mean',
    'mean2': 'mean',
    'dwelling_time3': 'mean',
    'sd3': 'mean',
    'mean3': 'mean',
    'label': 'sum'
}

summary_df = df.groupby(['transcript_id', 'transcript_position']).agg(agg_functions).reset_index()
summary_df['count'] = df.groupby(['transcript_id', 'transcript_position']).size().reset_index(name='count')['count']
summary_df['label_percentage'] = summary_df['label'] / summary_df['count'] * 100
summary_df = summary_df.rename(columns={'label': 'label_sum'})
summary_df['label'] = summary_df['label_percentage'].apply(lambda x: 0 if x == 0 else 1)

#summary_df.to_csv("summary_df.csv")
summary_df.to_pickle("/Users/tangyating/downloads/summary_df.pkl")

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

df = pd.read_pickle('/Users/tangyating/downloads/df.pkl')
count_df = df.groupby(['transcript_id', 'transcript_position']).size().reset_index(name='count')
df = df.merge(count_df, on=['transcript_id', 'transcript_position'])

summary_df = pd.read_pickle('/Users/tangyating/downloads/summary_df.pkl')

In [43]:
df

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,gene_id,label,count
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0,185
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0,185
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0,185
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0,185
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0,185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11027101,ENST00000641834,1693,TTGACAT,0.00418,7.49,108.0,0.00564,10.20,116.0,0.01000,2.01,76.4,ENSG00000167747,0,52
11027102,ENST00000641834,1693,TTGACAT,0.00664,1.91,109.0,0.00598,12.30,110.0,0.01760,2.61,74.6,ENSG00000167747,0,52
11027103,ENST00000641834,1693,TTGACAT,0.00721,4.58,105.0,0.00398,6.58,113.0,0.00316,2.28,85.3,ENSG00000167747,0,52
11027104,ENST00000641834,1693,TTGACAT,0.00266,2.33,109.0,0.00913,10.40,108.0,0.00664,4.44,76.8,ENSG00000167747,0,52


In [44]:
summary_df

,transcript_id,transcript_position,gene_id,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,label_sum,count,label_percentage,label
0,ENST00000000233,244,ENSG00000004059,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,4.386989,80.570270,0,185,0.0,0
1,ENST00000000233,261,ENSG00000004059,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,3.016599,94.290698,0,172,0.0,0
2,ENST00000000233,316,ENSG00000004059,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,2.087146,89.364324,0,185,0.0,0
3,ENST00000000233,332,ENSG00000004059,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,2.236520,89.154000,0,200,0.0,0
4,ENST00000000233,368,ENSG00000004059,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,4.260253,85.178788,0,198,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,ENSG00000167747,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,4.005616,82.004110,73,73,100.0,1
121834,ENST00000641834,1429,ENSG00000167747,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,3.644638,80.497101,0,69,0.0,0
121835,ENST00000641834,1531,ENSG00000167747,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,2.181563,84.190625,64,64,100.0,1
121836,ENST00000641834,1537,ENSG00000167747,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,2.540877,82.289474,0,57,0.0,0


In [3]:
# ===============================================================================================================================================================
# Using rus to deal with imbalanced dataset, Train and test data on df ==========================================================================================
# ===============================================================================================================================================================
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

combined = df
label_encoder = LabelEncoder()
combined['transcript_id'] = label_encoder.fit_transform(combined['transcript_id'])
combined['gene_id'] = label_encoder.fit_transform(combined['gene_id'])
combined = pd.get_dummies(combined, columns=['combined nucleotides'], prefix='nucleotide', drop_first=True)

X = combined.drop('label', axis=1)
y = combined['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

rus = RandomUnderSampler(random_state=42)
X_usampled, y_usampled = rus.fit_resample(X_train, y_train)

In [3]:
# XG Boost
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

best_params = {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
xgb_model = xgb.XGBClassifier(**best_params)
xgb_model.fit(X_usampled, y_usampled)
y_pred_xgb = xgb_model.predict(X_test)

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_test, y_pred_xgb)
confusion = confusion_matrix(y_test, y_pred_xgb)
classification_report_str = classification_report(y_test, y_pred_xgb)
roc_auc_xgb = roc_auc_score(y_test, y_pred_xgb)

print("Best Parameters:", best_params)
print("Accuracy_xgb:", accuracy)
print("Confusion Matrix_xgb:\n", confusion)
print("Classification Report_xgb:\n", classification_report_str)
print("ROC AUC Score:", roc_auc_xgb)

Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
Accuracy_xgb: 0.8247559877429353
Confusion Matrix_xgb:
 [[1732419  373088]
 [  13399   86516]]
Classification Report_xgb:
               precision    recall  f1-score   support

           0       0.99      0.82      0.90   2105507
           1       0.19      0.87      0.31     99915

    accuracy                           0.82   2205422
   macro avg       0.59      0.84      0.60   2205422
weighted avg       0.96      0.82      0.87   2205422

ROC AUC Score: 0.844349867684282


In [ ]:
import numpy as np
predictions_df_xgb = pd.DataFrame({'y_pred_xgb': y_pred_xgb})
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
predictions_df_xgb.reset_index(drop=True, inplace=True)

df_with_predictions_xgb = pd.concat([X_test, y_test, predictions_df_xgb], axis=1)

mean_predictions = df_with_predictions_xgb.groupby(['transcript_id', 'transcript_position'])['y_pred_xgb'].mean()
y_test_xgb = df_with_predictions_xgb.groupby(['transcript_id', 'transcript_position'])['label'].mean()
mean_predictions = mean_predictions.rename('y_pred_xgb_mean')
y_test_xgb = y_test_xgb.rename('y_test_xgb_mean')

mean_predictions_df = mean_predictions.reset_index()
y_test_xgb_df = y_test_xgb.reset_index()

result_df = pd.concat([mean_predictions_df, y_test_xgb_df['y_test_xgb_mean']], axis=1)
result_df

threshold = 0.5
result_df['y_pred_xgb_binary'] = np.where(result_df['y_pred_xgb_mean'] >= threshold, 1, 0)

roc_auc_xgb_agg = roc_auc_score(result_df['y_test_xgb_mean'], result_df['y_pred_xgb_binary'])
roc_auc_xgb_agg

0.8139094238044571

In [6]:
# Random Forest
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder

rf_model = RandomForestClassifier(random_state=42, n_estimators=100)

param_grid = {
    'max_depth': [10],
    'min_samples_split': [5],
}

rf_grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='roc_auc')
rf_grid_search.fit(X_usampled, y_usampled)

best_model = rf_grid_search.best_estimator_
best_params = rf_grid_search.best_params_

best_model.n_estimators = 100
y_pred_rf = best_model.predict_proba(X_test)[:, 1]

threshold = 0.5
y_pred_rf_binary = (y_pred_rf >= threshold).astype(int)

accuracy = accuracy_score(y_test, y_pred_rf_binary)
confusion = confusion_matrix(y_test, y_pred_rf_binary)
classification_report_str = classification_report(y_test, y_pred_rf_binary)
roc_auc_rf = roc_auc_score(y_test, y_pred_rf)

print("Best Hyperparameters:", best_params)
print("Accuracy_rf:", accuracy)
print("Confusion Matrix_rf:\n", confusion)
print("Classification Report_rf:\n", classification_report_str)
print("ROC AUC Score_rf:", roc_auc_rf)


Best Hyperparameters: {'max_depth': 10, 'min_samples_split': 5}
Accuracy_rf: 0.71952306633379
Confusion Matrix_rf:
 [[1509058  596449]
 [  22121   77794]]
Classification Report_rf:
               precision    recall  f1-score   support

           0       0.99      0.72      0.83   2105507
           1       0.12      0.78      0.20     99915

    accuracy                           0.72   2205422
   macro avg       0.55      0.75      0.52   2205422
weighted avg       0.95      0.72      0.80   2205422

ROC AUC Score_rf: 0.8294662862132984


In [7]:
import numpy as np
predictions_df_rf = pd.DataFrame({'y_pred_rf': y_pred_rf})
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
predictions_df_rf.reset_index(drop=True, inplace=True)

df_with_predictions_rf = pd.concat([X_test, y_test, predictions_df_rf], axis=1)

mean_predictions = df_with_predictions_rf.groupby(['transcript_id', 'transcript_position'])['y_pred_rf'].mean()
y_test_rf = df_with_predictions_rf.groupby(['transcript_id', 'transcript_position'])['label'].mean()
mean_predictions = mean_predictions.rename('y_pred_rf_mean')
y_test_rf = y_test_rf.rename('y_test_rf_mean')

mean_predictions_df = mean_predictions.reset_index()
y_test_rf_df = y_test_rf.reset_index()

result_df = pd.concat([mean_predictions_df, y_test_rf_df['y_test_rf_mean']], axis=1)
result_df

threshold = 0.5
result_df['y_pred_rf_binary'] = np.where(result_df['y_pred_rf_mean'] >= threshold, 1, 0)

roc_auc_rf_agg = roc_auc_score(result_df['y_test_rf_mean'], result_df['y_pred_rf_binary'])
roc_auc_rf_agg

0.7357229568359933

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

logistic_model = LogisticRegression()
logistic_model.fit(X_usampled, y_usampled)
y_pred_lg = logistic_model.predict_proba(X_test)[:, 1]

threshold = 0.5
y_pred_lg_binary = (y_pred_lg >= threshold).astype(int)

accuracy = accuracy_score(y_test, y_pred_lg_binary)
confusion = confusion_matrix(y_test, y_pred_lg_binary)
classification_report_str = classification_report(y_test, y_pred_lg_binary)

roc_auc = roc_auc_score(y_test, y_pred_lg)
fpr, tpr, _ = roc_curve(y_test, y_pred_lg)
precision, recall, _ = precision_recall_curve(y_test, y_pred_lg)

print("Accuracy_lg:", accuracy)
print("Confusion Matrix_lg:\n", confusion)
print("Classification Report_lg:\n", classification_report_str)
print("ROC AUC Score_lg:", roc_auc)

In [5]:
# Gradient Boosting (XGBoost) for Classification
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

xgb_model = xgb.XGBClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
}

xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='roc_auc', cv=5)
xgb_grid_search.fit(X_usampled, y_usampled)

best_model = xgb_grid_search.best_estimator_
best_params = xgb_grid_search.best_params_

y_pred_gb = best_model.predict_proba(X_test)[:, 1]

threshold = 0.5
y_pred_gb_binary = (y_pred_gb >= threshold).astype(int)

roc_auc_gb = roc_auc_score(y_test, y_pred_gb)
accuracy = accuracy_score(y_test, y_pred_gb_binary)
confusion = confusion_matrix(y_test, y_pred_gb_binary)
classification_report_str = classification_report(y_test, y_pred_gb_binary)

print("Best Hyperparameters:", best_params)
print("Accuracy_lg:", accuracy)
print("Confusion Matrix_lg:\n", confusion)
print("Classification Report_lg:\n", classification_report_str)
print("ROC AUC Score of Gradient Boosting (XGBoost):", roc_auc_gb)

Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
Accuracy_lg: 0.8247559877429353
Confusion Matrix_lg:
 [[1732419  373088]
 [  13399   86516]]
Classification Report_lg:
               precision    recall  f1-score   support

           0       0.99      0.82      0.90   2105507
           1       0.19      0.87      0.31     99915

    accuracy                           0.82   2205422
   macro avg       0.59      0.84      0.60   2205422
weighted avg       0.96      0.82      0.87   2205422

ROC AUC Score of Gradient Boosting (XGBoost): 0.9276959475222228


In [31]:
import numpy as np
predictions_df_gb = pd.DataFrame({'y_pred_gb': y_pred_gb})
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
predictions_df_gb.reset_index(drop=True, inplace=True)

df_with_predictions_gb = pd.concat([X_test, y_test, predictions_df_gb], axis=1)

mean_predictions = df_with_predictions_gb.groupby(['transcript_id', 'transcript_position'])['y_pred_gb'].mean()
y_test_gb = df_with_predictions_gb.groupby(['transcript_id', 'transcript_position'])['label'].mean()
mean_predictions = mean_predictions.rename('y_pred_gb_mean')
y_test_gb = y_test_gb.rename('y_test_gb_mean')

mean_predictions_df = mean_predictions.reset_index()
y_test_gb_df = y_test_gb.reset_index()

result_df = pd.concat([mean_predictions_df, y_test_gb_df['y_test_gb_mean']], axis=1)
result_df

threshold = 0.5
result_df['y_pred_gb_binary'] = np.where(result_df['y_pred_gb_mean'] >= threshold, 1, 0)

roc_auc_gb_agg = roc_auc_score(result_df['y_test_gb_mean'], result_df['y_pred_gb_binary'])

roc_auc_gb_agg

0.8151117365037728

In [9]:
# Neural Networks (Feedforward) for Classification
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold

def create_model(neurons_in_input=64, neurons_in_hidden=32, learning_rate=0.001):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(neurons_in_input, activation='relu', input_shape=(input_dim,), name='input_layer'),
        tf.keras.layers.Dense(neurons_in_hidden, activation='relu', name='hidden_layer'),
        tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy')
    return model

input_dim = 300
keras_classifier = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {
    'neurons_in_input': [64, 128],
    'neurons_in_hidden': [32, 64],
    'learning_rate': [0.001, 0.01]
}

grid_search = GridSearchCV(keras_classifier, param_grid, cv=StratifiedKFold(n_splits=3), scoring='roc_auc')
grid_search.fit(X_usampled, y_usampled, epochs=10, batch_size=32)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred_nn = best_model.predict(X_test).flatten()

threshold = 0.5
y_pred_nn_binary = (y_pred_nn >= threshold).astype(int)

roc_auc_nn = roc_auc_score(y_test, y_pred_nn)
accuracy = accuracy_score(y_test, y_pred_nn_binary)
confusion = confusion_matrix(y_test, y_pred_nn_binary)
classification_report_str = classification_report(y_test, y_pred_nn_binary)

print("Best Hyperparameters:", best_params)
print("Accuracy_nn:", accuracy)
print("Confusion Matrix_nn:\n", confusion)
print("Classification Report_nn:\n", classification_report_str)
print("ROC AUC Score of Neural Networks:", roc_auc_nn)

/var/folders/lr/2nfz4tks7jvdbfs0vc58q0x00000gn/T/ipykernel_41847/2298579575.py:20: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_classifier = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)
2023-10-22 13:40:47.764226: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import numpy as np
predictions_df_nn = pd.DataFrame({'y_pred_nn': y_pred_nn})
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
predictions_df_nn.reset_index(drop=True, inplace=True)

df_with_predictions_nn = pd.concat([X_test, y_test, predictions_df_nn], axis=1)

mean_predictions = df_with_predictions_nn.groupby(['transcript_id', 'transcript_position'])['y_pred_nn'].mean()
y_test_nn = df_with_predictions_nn.groupby(['transcript_id', 'transcript_position'])['label'].mean()
mean_predictions = mean_predictions.rename('y_pred_nn_mean')
y_test_nn = y_test_nn.rename('y_test_nn_mean')

mean_predictions_df = mean_predictions.reset_index()
y_test_nn_df = y_test_nn.reset_index()

result_df = pd.concat([mean_predictions_df, y_test_nn_df['y_test_nn_mean']], axis=1)
result_df

threshold = 0.5
result_df['y_pred_nn_binary'] = np.where(result_df['y_pred_nn_mean'] >= threshold, 1, 0)

roc_auc_nn_agg = roc_auc_score(result_df['y_test_nn_mean'], result_df['y_pred_nn_binary'])
roc_auc_nn_agg

In [55]:
# ===============================================================================================================================================================
# Using rus to deal with imbalanced dataset, Train and test data on summary_df ==================================================================================
# ===============================================================================================================================================================
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

combined = summary_df
label_encoder = LabelEncoder()
combined['transcript_id'] = label_encoder.fit_transform(combined['transcript_id'])
combined['gene_id'] = label_encoder.fit_transform(combined['gene_id'])
combined = pd.get_dummies(combined, columns=['combined nucleotides'], prefix='nucleotide', drop_first=True)

X = combined.drop(['label', 'label_percentage', 'label_sum'], axis=1)
y = combined['label']
X_train_summary, X_test_summary, y_train_summary, y_test_summary = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

rus = RandomUnderSampler(random_state=42)
X_usampled_summary, y_usampled_summary = rus.fit_resample(X_train_summary, y_train_summary)

In [17]:
# XG Boost
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

xgb_model = xgb.XGBClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
}

xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='roc_auc', cv=5)
xgb_grid_search.fit(X_usampled_summary, y_usampled_summary)

best_params = xgb_grid_search.best_params_
best_model = xgb_grid_search.best_estimator_

y_pred_xgb = best_model.predict(X_test_summary)

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_test_summary, y_pred_xgb)
confusion = confusion_matrix(y_test_summary, y_pred_xgb)
classification_report_str = classification_report(y_test_summary, y_pred_xgb)
roc_auc_xgb = roc_auc_score(y_test_summary, y_pred_xgb)

print("Best Parameters:", best_params)
print("Accuracy_xgb:", accuracy)
print("Confusion Matrix_xgb:\n", confusion)
print("Classification Report_xgb:\n", classification_report_str)
print("ROC AUC Score:", roc_auc_xgb)

Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
Accuracy_xgb: 0.8294484569927774
Confusion Matrix_xgb:
 [[19349  3924]
 [  232   863]]
Classification Report_xgb:
               precision    recall  f1-score   support

           0       0.99      0.83      0.90     23273
           1       0.18      0.79      0.29      1095

    accuracy                           0.83     24368
   macro avg       0.58      0.81      0.60     24368
weighted avg       0.95      0.83      0.88     24368

ROC AUC Score: 0.8097602273746185


In [22]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = RandomForestClassifier(random_state=42)
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='roc_auc', cv=5)

rf_grid_search.fit(X_usampled_summary, y_usampled_summary)

best_params = rf_grid_search.best_params_
best_rf_model = rf_grid_search.best_estimator_

y_pred_rf = best_rf_model.predict_proba(X_test_summary)[:, 1]

threshold = 0.5
y_pred_rf_binary = (y_pred_rf >= threshold).astype(int)

accuracy = accuracy_score(y_test_summary, y_pred_rf_binary)
confusion = confusion_matrix(y_test_summary, y_pred_rf_binary)
classification_report_str = classification_report(y_test_summary, y_pred_rf_binary)
roc_auc_rf_summary = roc_auc_score(y_test_summary, y_pred_rf)

print("Best Hyperparameters:", best_params)
print("Accuracy_rf:", accuracy)
print("Confusion Matrix_rf:\n", confusion)
print("Classification Report_rf:\n", classification_report_str)
print("ROC AUC Score_rf:", roc_auc_rf_summary)

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Accuracy_rf: 0.8107353906762967
Confusion Matrix_rf:
 [[18870  4403]
 [  209   886]]
Classification Report_rf:
               precision    recall  f1-score   support

           0       0.99      0.81      0.89     23273
           1       0.17      0.81      0.28      1095

    accuracy                           0.81     24368
   macro avg       0.58      0.81      0.58     24368
weighted avg       0.95      0.81      0.86     24368

ROC AUC Score_rf: 0.8933163579329487


In [56]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
}

logistic_model = LogisticRegression()
lg_grid_search = GridSearchCV(estimator=logistic_model, param_grid=param_grid, scoring='roc_auc', cv=5)

lg_grid_search.fit(X_usampled_summary, y_usampled_summary)

best_params = lg_grid_search.best_params_
best_lg_model = lg_grid_search.best_estimator_

y_pred_lg = best_lg_model.predict_proba(X_test_summary)[:, 1]

threshold = 0.5
y_pred_lg_binary = (y_pred_lg >= threshold).astype(int)

accuracy = accuracy_score(y_test_summary, y_pred_lg_binary)
confusion = confusion_matrix(y_test_summary, y_pred_lg_binary)
classification_report_str = classification_report(y_test_summary, y_pred_lg_binary)
roc_auc_lg_summary = roc_auc_score(y_test_summary, y_pred_lg)

print("Best Hyperparameters:", best_params)
print("Accuracy_lg:", accuracy)
print("Confusion Matrix_lg:\n", confusion)
print("Classification Report_lg:\n", classification_report_str)
print("ROC AUC Score_lg:", roc_auc_lg_summary)

/Users/tangyating/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/tangyating/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

Best Hyperparameters: {'C': 0.1, 'penalty': 'l2'}
Accuracy_lg: 0.6139609323703218
Confusion Matrix_lg:
 [[14239  9034]
 [  373   722]]
Classification Report_lg:
               precision    recall  f1-score   support

           0       0.97      0.61      0.75     23273
           1       0.07      0.66      0.13      1095

    accuracy                           0.61     24368
   macro avg       0.52      0.64      0.44     24368
weighted avg       0.93      0.61      0.72     24368

ROC AUC Score_lg: 0.6694535596641571


/Users/tangyating/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
# Gradient Boosting (XGBoost) for Classification
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01],
}

xgb_model = xgb.XGBClassifier(random_state=42)
xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='roc_auc', cv=5)
xgb_grid_search.fit(X_usampled_summary, y_usampled_summary)

best_params = xgb_grid_search.best_params_
best_xgb_model = xgb_grid_search.best_estimator_

y_pred_gb = best_xgb_model.predict_proba(X_test_summary)[:, 1]

threshold = 0.5
y_pred_gb_binary = (y_pred_gb >= threshold).astype(int)

roc_auc_gb_summary = roc_auc_score(y_test_summary, y_pred_gb)
accuracy = accuracy_score(y_test_summary, y_pred_gb_binary)
confusion = confusion_matrix(y_test_summary, y_pred_gb_binary)
classification_report_str = classification_report(y_test_summary, y_pred_gb_binary)

print("Best Hyperparameters:", best_params)
print("Accuracy_xgb:", accuracy)
print("Confusion Matrix_xgb:\n", confusion)
print("Classification Report_xgb:\n", classification_report_str)
print("ROC AUC Score of Gradient Boosting (XGBoost):", roc_auc_gb_summary)

Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Accuracy_xgb: 0.8262065003282995
Confusion Matrix_xgb:
 [[19261  4012]
 [  223   872]]
Classification Report_xgb:
               precision    recall  f1-score   support

           0       0.99      0.83      0.90     23273
           1       0.18      0.80      0.29      1095

    accuracy                           0.83     24368
   macro avg       0.58      0.81      0.60     24368
weighted avg       0.95      0.83      0.87     24368

ROC AUC Score of Gradient Boosting (XGBoost): 0.8933697837480751


In [21]:
# Neural Networks (Feedforward) for Classification
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

def create_model(neurons_in_input, neurons_in_hidden, learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(neurons_in_input, activation='relu', input_shape=(input_dim,), name='input_layer'),
        tf.keras.layers.Dense(neurons_in_hidden, activation='relu', name='hidden_layer'),
        tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy')
    return model

input_dim = 300
X_train, X_test, y_train, y_test = train_test_split(X_usampled_summary, y_usampled_summary, test_size=0.2, random_state=42)

param_grid = {
    'neurons_in_input': [64, 128],
    'neurons_in_hidden': [32, 64],
    'learning_rate': [0.001, 0.01]
}

keras_classifier = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)

grid_search = GridSearchCV(keras_classifier, param_grid, cv=3, scoring='roc_auc')
grid_search.fit(X_train, y_train, epochs=10, batch_size=32)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred_nn_summary = best_model.predict(X_test).flatten()

threshold = 0.5
y_pred_nn_binary_summary = (y_pred_nn_summary >= threshold).astype(int)

roc_auc_nn_summary = roc_auc_score(y_test, y_pred_nn_summary)
accuracy = accuracy_score(y_test, y_pred_nn_binary_summary)
confusion = confusion_matrix(y_test, y_pred_nn_binary_summary)
classification_report_str = classification_report(y_test, y_pred_nn_binary_summary)

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_report_str)
print("ROC AUC Score of Neural Networks:", roc_auc_nn_summary)

/var/folders/lr/2nfz4tks7jvdbfs0vc58q0x00000gn/T/ipykernel_32921/1513607251.py:28: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_classifier = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)


Best Hyperparameters: {'learning_rate': 0.001, 'neurons_in_hidden': 32, 'neurons_in_input': 64}
Accuracy: 0.5799086757990868
Confusion Matrix:
 [[748 122]
 [614 268]]
Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.86      0.67       870
           1       0.69      0.30      0.42       882

    accuracy                           0.58      1752
   macro avg       0.62      0.58      0.55      1752
weighted avg       0.62      0.58      0.54      1752

ROC AUC Score of Neural Networks: 0.5818124951129877
